# Jupyter notebook to find the seismic activity on the Moon

# Read the Apollo 12 Grade A catalog

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

Let's take a look at the training data for the lunar dataset. In addition to the data itself, we include a catalog that will tell you which events happen when in the data. The catalog includes the name of the file, the absolute time, the relative time in seconds (relative to the start of the file), the event ID (evid), and the type of moonquake. The types of moonquakes include impacts, deep moonquakes, and shallow moonquakes. You do not have to worry about predicting the type of moonquakes, that's just fun information for you to know! 

**Note**: For your prediction, feel free to include either the absolute time or relative time, just make sure to mark it using the same header in the CSV file so we can easily score it!

In [2]:
cat_directory = './data/lunar/training/catalogs/'
cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
cat = pd.read_csv(cat_file)
cat

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq
...,...,...,...,...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156,1974-10-14T17:43:00.000000,63780.0,evid00156,impact_mq
72,xa.s12.00.mhz.1975-04-12HR00_evid00191,1975-04-12T18:15:00.000000,65700.0,evid00191,impact_mq
73,xa.s12.00.mhz.1975-05-04HR00_evid00192,1975-05-04T10:05:00.000000,36300.0,evid00192,impact_mq
74,xa.s12.00.mhz.1975-06-24HR00_evid00196,1975-06-24T16:03:00.000000,57780.0,evid00196,impact_mq


# Sample short-term average / long-term average (STA/LTA) detection algorithm

A STA/LTA algorithm moves two time windows of two lengths (one short, one long) across the seismic data. The algorithm calculates the average amplitude in both windows, and calculates the ratio between them. If the data contains an earthquake, then the short-term window containing the earthquake will be much larger than the long-term window -- resulting in a detection. 

Next, we define the values of the characteristic function (i.e. amplitude ratio between short-term and long-term windows) where we flag a seismic detection. These values are called triggers. There are two types of triggers -- "on" and "off", defined as follows:

1. "on" : If the characteristic function is above this value, then a seismic event begins. 
2. "off" : If the characteristic function falls below this value (after an "on" trigger), than a seismic event ends. 

In [3]:
from obspy.signal.invsim import cosine_taper
from obspy.signal.filter import highpass
from obspy.signal.trigger import classic_sta_lta, plot_trigger, trigger_onset
from scipy.optimize import differential_evolution
from glob import glob

def read_sample(mseed_file):
    st = read(mseed_file)
    tr = st.traces[0].copy()
    return tr

def run_sta_lta_algorithm(sta_len, lta_len, thr_on, thr_off, df, tr_data):
    # Run Obspy's STA/LTA to obtain a characteristic function
    # This function basically calculates the ratio of amplitude between the short-term 
    # and long-term windows, moving consecutively in time across the data
    cft = classic_sta_lta(tr_data, int(sta_len * df), int(lta_len * df))
    
    # The first column contains the indices where the trigger is turned "on". 
    # The second column contains the indices where the trigger is turned "off".
    on_off = np.array(trigger_onset(cft, thr_on, thr_off))
    return on_off

def min_func(values, df, tr_data):
    sta_len, lta_len_coeff, thr_on, thr_off_coeff = values
    lta_len = sta_len * lta_len_coeff
    thr_off = thr_on * thr_off_coeff
    on_off = run_sta_lta_algorithm(sta_len, lta_len, thr_on, thr_off, df, tr_data)
    value = abs(len(on_off) - 1)
    return value

def run_algorithm(filepath):
    fnames, time_rels, detection_times = [], [], []
    print('Index, filename, sta_len, lta_len, thr_on, thr_off, loss, value.')
    
    for i, filename in enumerate(sorted(glob(filepath))):
        tr = read_sample(filename)
    
        # Sampling frequency of our trace
        df = tr.stats.sampling_rate
    
        tr_times = tr.times()
        tr_data = tr.data
        
        bounds = [(10, 1000), (1.05, 5.0), (3.5, 4.5), (0.05, 0.5)]
        result = differential_evolution(min_func, bounds, args=(df, tr_data))
        
        loss_value = result.fun
    
        if loss_value == 0.0:
            sta_len, lta_len_coeff, thr_on, thr_off_coeff = result.x
            lta_len = sta_len * lta_len_coeff
            thr_off = thr_on * thr_off_coeff
        else:
            # How long should the short-term and long-term window be, in seconds.
            sta_len = 120
            lta_len = 600
        
            # Play around with the on and off triggers, based on values in the characteristic function.
            thr_on = 4.0
            thr_off = 1.5
    
        on_off = run_sta_lta_algorithm(sta_len, lta_len, thr_on, thr_off, df, tr_data)
        
        if len(on_off) > 0:
            triggers = on_off[0]
            time_rel = tr_times[triggers[0]]
        else:
            time_rel = 0.0

        starttime = tr.stats.starttime.datetime
        on_time = starttime + timedelta(seconds=time_rel)
        on_time_str = datetime.strftime(on_time, '%Y-%m-%dT%H:%M:%S.%f')

        fnames.append(filename)
        detection_times.append(on_time_str)
        time_rels.append(time_rel)
        
        print(f'{i + 1} {filename} {sta_len:.2f} {lta_len:.2f} {thr_on:.2f} {thr_off:.2f} {loss_value} {len(on_off)}')

    return fnames, time_rels, detection_times

fnames, time_rels, detection_times = run_algorithm('./data/lunar/training/data/S12_GradeA/*.mseed')

Index, filename, sta_len, lta_len, thr_on, thr_off, loss, value.
1 ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-01-19HR00_evid00002.mseed 120.00 600.00 4.00 1.50 1.0 19
2 ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-03-25HR00_evid00003.mseed 598.25 2950.38 3.78 0.69 0.0 1
3 ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-03-26HR00_evid00004.mseed 146.16 615.15 3.94 1.28 0.0 1
4 ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-04-25HR00_evid00006.mseed 382.44 1504.11 3.61 0.47 0.0 1
5 ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-04-26HR00_evid00007.mseed 357.68 1722.66 3.60 1.36 0.0 1
6 ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-06-15HR00_evid00008.mseed 244.49 1058.61 4.25 0.24 0.0 1
7 ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-06-26HR00_evid00009.mseed 930.02 4328.81 4.14 1.65 0.0 1
8 ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-07-20HR00_evid00010.mseed 751.79 2765.06 3.62 1.21 0.0 1
9 ./data

**Note**: You do not have to worry about marking the end of the seismic trace (as you can see, even for us it's not very accurate!). For this challenge, all we care about is the start of the seismic waveform.

## Sample detection export into a catalog! 
There are many ways to do this, but we'll show a way to do it using pandas. 

In [4]:
fnames, time_rels, detection_times = run_algorithm('./data/lunar/test/data/**/*.mseed')
    
# Compile dataframe of detections
detect_df = pd.DataFrame(data = {'filename': fnames, 'time_abs(%Y-%m-%dT%H:%M:%S.%f)': detection_times, 'time_rel(sec)': time_rels})
detect_df.head()

Index, filename, sta_len, lta_len, thr_on, thr_off, loss, value.
1 ./data/lunar/test/data/S12_GradeB/xa.s12.00.mhz.1969-12-16HR00_evid00006.mseed 198.01 513.21 3.79 0.91 0.0 1
2 ./data/lunar/test/data/S12_GradeB/xa.s12.00.mhz.1970-01-09HR00_evid00007.mseed 419.84 2041.01 3.85 1.30 0.0 1
3 ./data/lunar/test/data/S12_GradeB/xa.s12.00.mhz.1970-02-07HR00_evid00014.mseed 373.59 1662.59 3.85 0.83 0.0 1
4 ./data/lunar/test/data/S12_GradeB/xa.s12.00.mhz.1970-02-18HR00_evid00016.mseed 814.82 3034.20 3.60 0.79 0.0 1
5 ./data/lunar/test/data/S12_GradeB/xa.s12.00.mhz.1970-03-14HR00_evid00018.mseed 89.08 350.54 3.60 0.98 0.0 1
6 ./data/lunar/test/data/S12_GradeB/xa.s12.00.mhz.1970-03-30HR00_evid00020.mseed 546.51 2598.69 3.72 1.49 0.0 1
7 ./data/lunar/test/data/S12_GradeB/xa.s12.00.mhz.1970-04-03HR00_evid00021.mseed 522.85 2143.78 3.64 1.16 0.0 1
8 ./data/lunar/test/data/S12_GradeB/xa.s12.00.mhz.1970-05-20HR00_evid00026.mseed 739.42 2707.07 3.54 1.17 0.0 1
9 ./data/lunar/test/data/S12_GradeB/xa.s12

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec)
0,./data/lunar/test/data/S12_GradeB/xa.s12.00.mh...,1969-12-16T22:16:07.725170,80167.547170
1,./data/lunar/test/data/S12_GradeB/xa.s12.00.mh...,1970-01-09T17:32:23.220340,63143.094340
2,./data/lunar/test/data/S12_GradeB/xa.s12.00.mh...,1970-02-07T18:56:16.118170,68175.547170
3,./data/lunar/test/data/S12_GradeB/xa.s12.00.mh...,1970-02-18T12:21:15.620698,44475.471698
4,./data/lunar/test/data/S12_GradeB/xa.s12.00.mh...,1970-03-14T14:49:14.633208,53354.113208


This can then be exported to a csv.

In [5]:
detect_df.to_csv('output/catalog.csv', index=False)

# Thank you very much for being a part of this challenge! Good luck!!!